In [1]:
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt  

import pykat.optics.maps as pkm
from pykat import finesse                 
from pykat.commands import *               
from pykat.optics.maps import *            
from IPython.display import display, HTML
from scipy.special import factorial
import fileinput

%matplotlib inline
pykat.init_pykat_plotting(dpi=90)

                                              ..-
    PyKat 1.2.1           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



## Content:
This file is about generating maps with increasing Zernike 22 content.

1. The original map is 'xITM_11_R1_Figure_finesse.txt' because its HG33 loss is ~0.2, which gives enough 'room' for this 'adding more astigmatism to reduce HG33 loss' method to work(unlike other measured maps the losses are either too small or too large).

2. The map 'xITM_11_R1_Figure_finesse.txt' is of shape 1133*1133 but both the zernike maps and random maps are of shape 1131*1131. So need to create zernike basis of shape 1133*1133, see below.

3. Create a map 'mapwithoutZ22.txt' from 'xITM_11_R1_Figure_finesse' after getting rid of the Zernike 22, which will be used later when using different radius of curvature in the x and y to represent the astigmatism.

In [2]:
# Generate Zernike basis of shape(1133*1133)
def Znm(shape, radius, step_size, n, m): 
    center = (shape-1)/2
    rrange = radius/step_size
    
    def theta(x,y):
        phi = np.arctan2(y, x)
        return phi

    def radial(x,y,n,m):
        if m<0:
            m=-m
        sum=0
        for k in range(int((n-m)/2)+1):   
            r=(-1)**k*factorial(n-k)/factorial(k)/factorial((n+m)/2-k)/factorial((n-m)/2-k)*((x**2+y**2)/(rrange**2))**(n/2-k)
            sum+=r
        return sum

    def angular(x,y,n,m): 
        a=theta(x,y)
        if m>=0:
            angular=np.cos(m*a)
        else:
            angular=-np.sin(m*a)
        return angular

    stepRange = np.arange(shape)-center
    x,y=np.meshgrid(stepRange,stepRange,sparse=True)
    zfunc=radial(x,y,n,m)*angular(x,y,n,m)
    for i in range(shape):
        for j in range(shape): 
            if (i-center)**2+(j-center)**2 >= rrange**2:
                zfunc[i][j]=0
    zmap=zfunc/np.abs(zfunc).max()

    return zmap

Z22=Znm(1133,0.15,0.0002667510125320405,2,2)

In [4]:
filename='aLIGOmeasuredmaps/xITM_11_R1_Figure_finesse.txt'
aLIGO=pd.read_csv(filename, header=None, sep=" ", skiprows=9).dropna(axis=1).values

coeff=(aLIGO*Z22).sum()/((Z22**2).sum())
print(f'The Z22 content in the original aLIGO map is {coeff} nm')

The Z22 content in the original aLIGO map is 0.12014186205896264 nm


In [5]:
filename='aLIGOmeasuredmaps/xITM_11_R1_Figure_finesse.txt'
aLIGO=pd.read_csv(filename, header=None, sep=" ", skiprows=9).dropna(axis=1).values

# Function that add more Z22 to the mapfile
def insert(originalfile,string): # Add the header before the mapdata
    with open(originalfile,'r') as f:
        with open('temp.txt','w') as f2: 
            f2.write(string)
            f2.write(f.read())
    os.rename('temp.txt',originalfile)

with open('aLIGOmeasuredmaps/xITM_11_R1_Figure_finesse.txt') as myfile: # take the header from the aLIGO measured map
    head = [next(myfile) for x in range(9)]
    header="".join(head)

def AddZ22(value, filename):
    mapData = aLIGO + value*Z22
    np.savetxt(filename, mapData, delimiter=' ',fmt='%1.6f')
    insert(filename, f'''{header}''') 

In [6]:
# The map after getting rid of Z22 in 'xITM_11_R1_Figure_finesse', called mapwithoutZ22
restData = aLIGO - Z22*0.12014186205896284 
np.savetxt('mapwithoutZ22.txt', restData, delimiter=' ',fmt='%1.6f')
insert('mapwithoutZ22.txt', f'''{header}''') 

# check the A22 is indeed zero in 'mapwithoutZ22'
Mapdata = pd.read_csv('mapwithoutZ22.txt', header=None, sep=" ", skiprows=9).dropna(axis=1).values
coeff = (Mapdata*Z22).sum()/((Z22**2).sum())
coeff

-2.8680541594027596e-10

In [19]:
i = 0
for value in np.linspace(0, 100, 44):
    AddZ22(value=value, filename=f'Z22maps/Z22map{i}.txt')
    i += 1